In [1]:
import numpy as np
import os
from PIL import Image
import helper
import json
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# training data path

train_normal_path = "./../../../datasets/rosacea-detection/train/normal"
train_rosacea_path = "./../../../datasets/rosacea-detection/train/rosacea"

In [3]:
X_normal = np.zeros((512 * 512 * 3, 500))
X_rosacea = np.zeros((512 * 512 * 3, 250))

print("Loading X_normal....")
for i, filename in zip(range(500),os.listdir(train_normal_path)):
    filepath = os.path.join(train_normal_path, filename)
    img = (np.array(Image.open(filepath)) ).flatten()
    X_normal[:, i] = img.copy()
    # print(f"{i}th normal loaded")
print(f"X_normal loaded, shape of {np.shape(X_normal)}")

print("Loading X_rosacea....")
for i, filename in zip(range(250),os.listdir(train_rosacea_path)):
    filepath = os.path.join(train_rosacea_path, filename)
    img = (np.array(Image.open(filepath)) ).flatten()
    X_rosacea[:, i] = img.copy()
    # print(f"{i}th rosacea loaded")
print(f"X_rosacea loaded, shape of {np.shape(X_rosacea)}")

Loading X_normal....
X_normal loaded, shape of (786432, 500)
Loading X_rosacea....
X_rosacea loaded, shape of (786432, 250)


In [4]:
X_train = np.hstack((X_normal, X_rosacea)).transpose()
print(np.shape(X_train))

(750, 786432)


In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)

In [6]:
Y_train = np.zeros(750)
Y_train[500:] = 1
# normal is 0
# rosacea is 1

In [7]:
# supplement experiment
val_normal_path = "./../../../datasets/rosacea-detection/val/normal"
val_rosacea_path = "./../../../datasets/rosacea-detection/val/rosacea"
for k in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15}:
    knn = KNeighborsClassifier(n_neighbors = k, metric = 'cosine')
    knn.fit(X_train_std, Y_train)
    TP_v, TN_v, FP_v, FN_v = 0, 0, 0, 0
     # Validation
    for filename in os.listdir(val_normal_path):
        file_path = os.path.join(val_normal_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            TN_v += 1
        else:
            FP_v += 1

    for filename in os.listdir(val_rosacea_path):
        file_path = os.path.join(val_rosacea_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            FN_v += 1
        else:
            TP_v += 1
            
    accuracy_v, precision_v, recall_v, F1_v = helper.get_performance_metrics(
    TP_v, TN_v, FP_v, FN_v)

    result = {
        "unequal" : 1,
        "n_neighbors" :k,
        "metric" : "cosine",
        "p" : None,
        "TN_v" : TN_v,
        "TP_v" : TP_v,
        "FP_v" : FP_v,
        "FN_v" : FN_v,
        "accuracy_v" : accuracy_v,
        "precision_v" : precision_v,
        "recall_v" : recall_v,
        "F1_v" : F1_v,

        "supplement":1
    }

    json_object = json.dumps(result, indent = 4)

    # writing to file
    json_name = f"knn-metric-cosine-n_neighbors={k}-supplement-val.json"
    json_path = os.path.join("./../results/knn-metric-cosine",json_name)
    with open(json_path,'w') as outfile:
        outfile.write(json_object)


MemoryError: Unable to allocate 4.39 GiB for an array with shape (750, 786432) and data type float64

In [ ]:
val_normal_path = "./../../../datasets/rosacea-detection/val/normal"
val_rosacea_path = "./../../../datasets/rosacea-detection/val/rosacea"
test_normal_path = "./../../../datasets/rosacea-detection/test/normal"
test_rosacea_path = "./../../../datasets/rosacea-detection/test/rosacea"

for k in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15}:
    knn = KNeighborsClassifier(n_neighbors = k, metric = 'cosine')
    knn.fit(X_train_std, Y_train)
    TP_v, TN_v, FP_v, FN_v = 0, 0, 0, 0
    TP_t, TN_t, FP_t, FN_t = 0, 0, 0, 0
    count = 0
    print(f"k is {k}")
    # Validation
    for filename in os.listdir(val_normal_path):
        file_path = os.path.join(val_normal_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            TN_v += 1
        else:
            FP_v += 1
        count += 1
        print(count)

    for filename in os.listdir(val_rosacea_path):
        file_path = os.path.join(val_rosacea_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            FN_v += 1
        else:
            TP_v += 1
        count += 1
        print(count)
        
    # Test
    for filename in os.listdir(test_normal_path):
        file_path = os.path.join(test_normal_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            TN_t += 1
        else:
            FP_t += 1
        count += 1
        print(count)
        
    for filename in os.listdir(test_rosacea_path):
        file_path = os.path.join(test_rosacea_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            FN_t += 1
        else:
            TP_t += 1
        count += 1
        print(count)
    # Output result
    accuracy_v, precision_v, recall_v, F1_v = helper.get_performance_metrics(
        TP_v, TN_v, FP_v, FN_v)
    
    accuracy_t, precision_t, recall_t, F1_t = helper.get_performance_metrics(
        TP_t, TN_t, FP_t, FN_t)

    result = {
        "unequal" : 1,
        "n_neighbors" :k,
        "metric" : "cosine",
        "p" : None,
        "TN_v" : TN_v,
        "TP_v" : TP_v,
        "FP_v" : FP_v,
        "FN_v" : FN_v,
        "accuracy_v" : accuracy_v,
        "precision_v" : precision_v,
        "recall_v" : recall_v,
        "F1_v" : F1_v,

        "TN_t" : TN_t,
        "TP_t" : TP_t,
        "FP_t" : FP_t,
        "FN_t" : FN_t,
        "accuracy_t" : accuracy_t,
        "precision_t" : precision_t,
        "recall_t" : recall_t,
        "F1_t" : F1_t
    }

    json_object = json.dumps(result, indent = 4)

    # writing to file
    # json_name = "p="+str(p)+"-n_neighbors=" + str(k)+".json"
    json_name = f"knn-metric-cosine-n_neighbors={k}.json"
    json_path = os.path.join("./../results",json_name)
    with open(json_path,'w') as outfile:
        outfile.write(json_object)


k is 1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2